In [21]:
from création_data import param_Dataset
import fonction_model as f
from IPython.display import display, Audio
from scipy.io.wavfile import write
import pickle

In [22]:
Fe = 2**15

## Medium 1:

In [25]:
Fichier = "1ier_data_set/medium_1/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = True, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = False)

In [26]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['nu_table'] = param_dataset[i][6]
    dico[kle]['E_table'] = param_dataset[i][7]
    dico[kle]['rho_table'] = param_dataset[i][8]
    dico[kle]['Lx_table'] = param_dataset[i][9]
    dico[kle]['Ly_table'] = param_dataset[i][10]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## medium 2

In [ ]:
Fichier = "1ier_data_set/medium_2/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = True, 
                                             metal_T = False,
                                             plexi_T = False)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['nu_table'] = param_dataset[i][6]
    dico[kle]['E_table'] = param_dataset[i][7]
    dico[kle]['rho_table'] = param_dataset[i][8]
    dico[kle]['Lx_table'] = param_dataset[i][9]
    dico[kle]['Ly_table'] = param_dataset[i][10]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## Acier : 

In [23]:
Fichier = "1ier_data_set/acier/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = True,
                                             plexi_T = False)

In [24]:
for i in range(1):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

/var/folders/k2/m05vfpwn0qn71pjn0jnlb0l80000gn/T/ipykernel_45076/644401018.py:4: RuntimeWarning: invalid value encountered in divide
  write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))


In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['nu_table'] = param_dataset[i][6]
    dico[kle]['E_table'] = param_dataset[i][7]
    dico[kle]['rho_table'] = param_dataset[i][8]
    dico[kle]['Lx_table'] = param_dataset[i][9]
    dico[kle]['Ly_table'] = param_dataset[i][10]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)

## Plexi : 

In [ ]:
Fichier = "1ier_data_set/plexi/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = True)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))